<a href="https://colab.research.google.com/github/FelipeGBarreto/Projects/blob/master/Rascunho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASE – ANALISTA DE BUSINESS INTELLIGENCE:

A Fortbrasil tem como um dos seus maiores desafios a entrega de cartões na casa dos nossos clientes, visto que existem área de difícil acesso, outras que apenas o Correios opera, dentre outros. Tendo isso em vista, começamos a diversificar os modais de entrega dos cartões emitidos.

*O case consiste em criar um painel de acompanhamento dos cartões emitidos pela
empresa, e para isso estão sendo enviadas 05 bases:*

1. **“T_CARTÃO”:** Todos os cartões emitidos;
2. **“T_DESBLOQUEIO”:** Indica se o cliente desbloqueou o cartão, considerar apenas o primeiro
desbloqueio de cada ID_CARTÃO;
3. **“T_MODAIS”:** Indica por qual modal o cartão foi enviado e a data;
4. **“T_CANAL_DESBLOQUEIO”:** Registro de por onde foi feito o desbloqueio do cartão, caso não exista nenhum registro de desbloqueio nessa tabela, significa que ele foi feito via “Central de Atendimento”
5. **“T_COMPRAS”:** Registro de compras efetuadas em 2020, assim como seu status.
***


***Pede-se para:***

- Calcular o desbloqueio por safra (mês de emissão) dos cartões de 2020 – acompanhamento
acumulado por 90 dias desde a data de emissão;
- Calcular a ativação (primeira compra) por safra de emissão dos cartões de 2020 -
acompanhamento acumulado por 90 dias desde a data de envio;
- Analisar a efetividade de entrega (podendo ser medida pelo desbloqueio) e sugerir
melhorias;
- Analisar a performance de ativação e sugerir melhorias;

**OBS1:** devem ser considerados cartões válidos apenas os que não começam por ‘9099’ e que tem validade maior que 30 dias.

**OBS2:** use com inteligência as informações ofertadas para gerar um dashboard claro, gerencial e que possa ser usado para tomada de decisões. Gostamos de filtros que facilitem nossas análises.

**OBS3:** segue abaixo o exemplo de gráfico de desbloqueio/ativação acumulada X tempo que pode servir de base.

***



# Importando as Bibliotecas Necessárias

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

# Conhecendo os Dados

Conhecendo os dados e veridicando se há valores faltantes, bem como se há valores repetidos, com o fito de um melhor tratamento e precisão dos dados.

In [2]:
df_cartao = pd.read_csv("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_CARTAO.csv", sep=';')
df_desbloqueio = pd.read_excel("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_DESBLOQUEIO.xlsx")
df_modal = pd.read_excel("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_MODAIS.xlsx")
df_canal_desbloq = pd.read_csv("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_CANAL_DESBLOQUEIO.csv", sep=';')
df_compras = pd.read_csv("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_COMPRA.csv", sep=';')

In [16]:
t_cartao = df_cartao.copy()
t_desbloqueio = df_desbloqueio.copy()
t_modal = df_modal.copy()
t_canal_desbloq = df_canal_desbloq.copy()
t_compras = df_compras.copy()

In [17]:
# Criando uma função para entender os dados:
def analise_df(data):
  print("INFORMAÇÕES:\n"), display(data.head(3))
  print("\n"), display(data.info())
  print("\nLinhas e Colunas"), display(data.shape)
  print("\nValores Únicos:"), display(data.nunique().to_frame().T)
  print("\n-----------------\n\n")

In [ ]:
# Entendendo os dados
for i in [t_cartao,t_desbloqueio,t_modal,t_canal_desbloq,t_compras]:
  analise_df(i)

# Tratativa dos Dados

**Passo 1** - Retirar as horas das data, com o fito de maior precisão da análise

In [19]:
# Função para retirar as horas
def retirar_hr_datas(data,campo):
  data[campo] = data[campo].astype(str)
  separa_data = data[campo].str.split(" ", n = 1, expand = True)
  data[campo] = separa_data[0]
  data[campo] = pd.to_datetime(data[campo])

In [20]:
# Retirando as horas das datas de campos específicos
retirar_hr_datas(t_cartao,"DT_GERACAO_EMBOSSING")
retirar_hr_datas(t_cartao,"DT_VALIDADE")
# retirar_hr_datas(t_compras, "DT_COMPRA")

**Observações:**

- Para responder os dois primeiros pontos, não será necessário olhar para as tabelas t_modal e t_canal_desbloq.

**Passo 2** - Retirando os valores duplicados para responder os dois primeiros pontos, conforme orientações:

*`“T_DESBLOQUEIO”: Indica se o cliente desbloqueou o cartão, considerar apenas o primeiro desbloqueio de cada ID_CARTÃO`*

e

*`Calcular a ativação (primeira compra) por safra de emissão dos cartões de 2020 - acompanhamento acumulado por 90 dias desde a data de envio;`*

In [21]:
t_cartao = t_cartao.drop_duplicates()
t_desbloqueio = t_desbloqueio.sort_values("DT_DESBLOQUEIO").drop_duplicates(subset=["ID_CARTAO"], keep='first')
t_compras = t_compras.sort_values(["DT_COMPRA"]).drop_duplicates(subset=["ID_CARTAO"], keep='first')

**Passo 3** - Calculando a Safra de 2020

In [22]:
safra = t_cartao.copy()

# Cartões válidos 

## Retirar os que começa com 9099
nao_comeca_9099 = safra["DS_CARTAO"].str.split("*", n = 1, expand = True)
safra["NAO_COMECA_9099"] = nao_comeca_9099[0]
safra = safra.query("NAO_COMECA_9099!='9099'").drop(columns=["NAO_COMECA_9099"])

## Considerar somente os gerados (emitidos) em 2020
safra = safra.query("DT_GERACAO_EMBOSSING >= '2020-01-01' and DT_GERACAO_EMBOSSING < '2021-01-01'")
safra["Mes_Geracao"] = safra["DT_GERACAO_EMBOSSING"].dt.month

display(safra.head(2), safra.info(), safra.nunique().to_frame().T, safra.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586429 entries, 0 to 973233
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID_CARTAO             586429 non-null  int64         
 1   ID_CONTA              586429 non-null  int64         
 2   NU_SEQ_CARTAO         586429 non-null  int64         
 3   DT_GERACAO_EMBOSSING  586429 non-null  datetime64[ns]
 4   DT_VALIDADE           586429 non-null  datetime64[ns]
 5   DS_CARTAO             586429 non-null  object        
 6   Mes_Geracao           586429 non-null  int64         
dtypes: datetime64[ns](2), int64(4), object(1)
memory usage: 35.8+ MB


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,Mes_Geracao
0,4137125,1861370,2,2020-02-19,2025-02-13,8596********6981,2
1,4137126,1861371,1,2020-02-19,2025-02-13,8596********1863,2


None

,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,Mes_Geracao
0,586429,438657,255,366,164,10480,12


(586429, 7)

**Passo 4** - Calculando o Desbloqueio Safra

In [23]:
desbloqueio_safra = t_cartao.merge(t_desbloqueio, how='left', on='ID_CARTAO')
desbloqueio_safra = desbloqueio_safra[~desbloqueio_safra["DT_DESBLOQUEIO"].isna()]
desbloqueio_safra["Mes_Geracao"] = desbloqueio_safra["DT_GERACAO_EMBOSSING"].dt.month

display(desbloqueio_safra.head(2),desbloqueio_safra.shape,desbloqueio_safra.info(), desbloqueio_safra.ID_CARTAO.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371543 entries, 4 to 973229
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID_CARTAO             371543 non-null  int64         
 1   ID_CONTA              371543 non-null  int64         
 2   NU_SEQ_CARTAO         371543 non-null  int64         
 3   DT_GERACAO_EMBOSSING  371543 non-null  datetime64[ns]
 4   DT_VALIDADE           371543 non-null  datetime64[ns]
 5   DS_CARTAO             371543 non-null  object        
 6   DT_DESBLOQUEIO        371543 non-null  datetime64[ns]
 7   Mes_Geracao           371543 non-null  int64         
dtypes: datetime64[ns](3), int64(4), object(1)
memory usage: 25.5+ MB


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO,Mes_Geracao
4,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13,2
6,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13,2


(371543, 8)

None

371543

**Passo 4** - Calculando a Ativação Safra

In [24]:
# Verificando os status dos cartões ativados
t_compras.DS_STATUS.unique()

## Ou seja, vou considerar somente os status 'Processada' e 'Normal' como realmente efetivado

array(['Processada', 'Cancelada', 'Desfeita', 'Estornada',
       'Baixada por prazo', 'Cancelada Fraude Negada', 'Pendente',
       'Normal'], dtype=object)

In [25]:
ativacao_safra = desbloqueio_safra.merge(t_compras, how='left', on=['ID_CARTAO','ID_CONTA'])
ativacao_safra = ativacao_safra[~ativacao_safra["DT_COMPRA"].isna()]
ativacao_safra["Mes_Geracao"] = ativacao_safra["DT_GERACAO_EMBOSSING"].dt.month

## Status de cartões ativados
ativacao_safra = ativacao_safra.query("DS_STATUS=='Processada' or DS_STATUS=='Normal'")

display(ativacao_safra.head(2),ativacao_safra.shape,ativacao_safra.info(),ativacao_safra.ID_CARTAO.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265337 entries, 0 to 371542
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID_CARTAO             265337 non-null  int64         
 1   ID_CONTA              265337 non-null  int64         
 2   NU_SEQ_CARTAO         265337 non-null  int64         
 3   DT_GERACAO_EMBOSSING  265337 non-null  datetime64[ns]
 4   DT_VALIDADE           265337 non-null  datetime64[ns]
 5   DS_CARTAO             265337 non-null  object        
 6   DT_DESBLOQUEIO        265337 non-null  datetime64[ns]
 7   Mes_Geracao           265337 non-null  int64         
 8   DS_STATUS             265337 non-null  object        
 9   DT_COMPRA             265337 non-null  object        
dtypes: datetime64[ns](3), int64(4), object(3)
memory usage: 22.3+ MB


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO,Mes_Geracao,DS_STATUS,DT_COMPRA
0,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13,2,Processada,2020-03-19 11:13:00
1,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13,2,Processada,2020-02-13 12:52:00


(265337, 10)

None

265337

# Comparativo 

Safra x Desbloqueio x Ativação

In [26]:
t_safra = safra.groupby("Mes_Geracao")[["ID_CARTAO"]].count()
t_desbloqueio_safra = desbloqueio_safra.groupby("Mes_Geracao")[["ID_CARTAO"]].count()
t_ativacao_safra = ativacao_safra.groupby("Mes_Geracao")[["ID_CARTAO"]].count()

In [27]:
comparativo = t_safra.merge(t_desbloqueio_safra, how='left', on='Mes_Geracao')
comparativo = comparativo.merge(t_ativacao_safra, how='left', on='Mes_Geracao')
comparativo = comparativo.rename(columns={"ID_CARTAO_x":"Safra","ID_CARTAO_y":"Desbloqueio de Safra","ID_CARTAO":"Ativação de Safra"})
comparativo.columns.name = "Mês de Referência"
comparativo

Mês de Referência,Safra,Desbloqueio de Safra,Ativação de Safra
Mes_Geracao,,,
1,65189,40061,NaN
2,50698,34332,13821.0
3,51557,37967,29405.0
4,36167,25148,21494.0
5,35428,25320,22345.0
6,54101,38933,34644.0
7,43713,30177,26661.0
8,47744,31586,27622.0
9,38703,28335,24115.0


***
***
***

In [32]:
ativacao_safra

,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO,Mes_Geracao,DS_STATUS,DT_COMPRA
0,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13,2,Processada,2020-03-19 11:13:00
1,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13,2,Processada,2020-02-13 12:52:00
2,4137133,1861374,2,2020-02-19,2025-02-13,8596********0242,2020-02-13,2,Processada,2020-02-13 14:14:00
3,4137138,1479019,4,2020-02-13,2025-02-13,8596********2391,2020-02-28,2,Processada,2020-03-01 19:19:00
4,4137140,1861377,2,2020-02-19,2025-02-13,8596********9857,2020-03-05,2,Processada,2020-08-07 08:43:00
...,...,...,...,...,...,...,...,...,...,...
371538,4891009,2078887,2,2020-12-18,2020-12-19,9099********1289,2020-12-24,12,Processada,2020-12-18 15:57:00
371539,4891025,2078893,1,2020-12-18,2025-02-13,8596********9369,2020-12-27,12,Processada,2021-01-01 10:37:00
371540,4891041,2078898,1,2020-12-18,2025-02-13,8596********4775,2020-12-18,12,Processada,2020-12-18 16:04:00
371541,4891053,2078903,1,2020-12-18,2025-02-13,8596********5107,2020-12-18,12,Processada,2020-12-18 16:37:00
